In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline

# Markov Chain
[Markov Chain](https://en.wikipedia.org/wiki/Markov_chain) is a model that describe a sequence of event within a system. 
Normally, in real life, current event is higly influenced by what had occured in the past.  
However, markov chain is simple. The occurence of current event depends on only the previous event.
```
Pr( Xn+1 = x | X1 = x1, X2 = x2, …, Xn = xn) = Pr( Xn+1 = x | Xn = xn)
```

# Text Generation Using Markov Chain
Here, we are considering the **event** of text generation and that event is described by the occurences of 
words/tokens in the text itself. So, a sequence of words give rise to next words.  
**For example**  
I am -> I am a programmer  
I am -> I am nostalgic  
<br/>
The occurence of next word in the sequence solely depends on the probability. So, the process is vaguely as:
- get textual data
- preprocess text
- count occurence of pairs {for example count of (i, am), (i, love), ...}
- create probabilities of occurence of next word given current word (this is done by the count value in above step)
- for new sequence of words, go on computing probabilities and get next probable word

## Load Data

In [2]:
tweetcsv = "data/tweets/clean.csv"

In [3]:
df = pd.read_csv(tweetcsv)

In [4]:
df['created_at'] = pd.to_datetime(df['created_at'])

In [5]:
df.head()

,created_at,text
0,2014-06-07 04:58:36,what pride therein lied the salvated victim of...
1,2014-06-02 15:59:11,game of life: simple yet complex
2,2014-06-01 16:38:18,well well <SENTENCE> who do we have here <SENT...
3,2014-06-01 16:31:18,paradox indeed <SENTENCE> <EMOTICON>
4,2014-06-01 12:49:22,i exist <SENTENCE> therefore i may not exist


In [6]:
# check nulls
df.isnull().sum(), df.shape

(created_at      0
 text          297
 dtype: int64, (13422, 2))

In [7]:
df = df.dropna()

In [8]:
# check nulls
df.isnull().sum(), df.shape

(created_at    0
 text          0
 dtype: int64, (13125, 2))

## Sort by Time (earliest to latest)

In [9]:
df = df.sort_values(by='created_at')

In [10]:
df.head()

,created_at,text
4,2014-06-01 12:49:22,i exist <SENTENCE> therefore i may not exist
3,2014-06-01 16:31:18,paradox indeed <SENTENCE> <EMOTICON>
2,2014-06-01 16:38:18,well well <SENTENCE> who do we have here <SENT...
1,2014-06-02 15:59:11,game of life: simple yet complex
0,2014-06-07 04:58:36,what pride therein lied the salvated victim of...


## Load Text

In [11]:
text = "\n".join(df['text'].values.tolist()).strip()

In [12]:
text[:100]

'i exist <SENTENCE> therefore i may not exist\nparadox indeed <SENTENCE> <EMOTICON>\nwell well <SENTENC'

In [13]:
# We have already done preprocessing.
# clean.csv is already a cleaned data

## Create Pairs
Creat word pairs like (w1, w2), (w1, w3), (w4, w9),....

In [14]:
def create_pairs(text):
    tokens = text.split()
    return list(zip(tokens, tokens[1:]))

In [15]:
pairs = create_pairs(text)
pairs[:5]

[('i', 'exist'),
 ('exist', '<SENTENCE>'),
 ('<SENTENCE>', 'therefore'),
 ('therefore', 'i'),
 ('i', 'may')]

# Trie Data Structure

Trie is an efficient data structure created from hashmap/dictionary where each node points to the next node until we reach some kind of leaf. In our case, leaf means the numerical value for word pairs.
So, we keep on traversing the trie until hitting a numerical value.
Here, we just keep on increasing the count of word pairs for next word if the next word follows current word.

Reference:
- https://en.wikipedia.org/wiki/Trie



In [16]:
def build_trie(pairs):
    trie = {}
    for pair in pairs:
        a, b = pair
        if a not in trie:
            trie[a] = {}
        if b not in trie[a]:
            trie[a][b] = 1
        else:
            trie[a][b] += 1
    return trie

In [17]:
trie = build_trie(pairs)


## Build Probabilities

Since trie stores the count of (current word, next word), we normalize the frequency to get the probability.

In [18]:
def build_probabilities(trie):
    for word, following in trie.items():
        total = sum(following.values())
        for key in following:
            following[key] /= total
    return trie

In [19]:
trie = build_probabilities(trie)

# Generate Tweet

## Version 1

Here, we are going to introduce some small randomness. This is one way to introduce random factor into probabilities.
In this technique, we are simply going to count probabilities of transition word and check the total probability to some random factor.

In [20]:
def generate1(trie, initial_word, max_len=5, verbose=False):
    res = []
    word = initial_word
    while len(res) < max_len:
        if word not in trie:
            break
        transitions = trie[word]
        if verbose:
            print("Current word :: ", word)
            print("Transitions :: ", transitions)
        t = 0
        for w in transitions:
            p = transitions[w]
            t += p
            if t and (random.random() * t) < p:
                next_word = w
            if verbose:
                print(w, p)
        res.append(word)
        word = next_word
    return res

In [29]:
generated_words = generate1(trie, initial_word='i', max_len=20, verbose=False)

In [30]:
' '.join(generated_words)

'i am precrastinating <SENTENCE> <EMOTICON> <EMOTICON> then play tron game after ages ago total beauty of adhd perhaps <SENTENCE> freakingly'

## Version 2

This is another way to introduce some randomness to the states. Here we are going to normalize probabilities of transition words with random number and get the next state with max probability (just like in version 1)

In [31]:
def generate2(trie, initial_word, max_len=5, verbose=False):
    res = []
    word = initial_word
    while len(res) < max_len:
        if word not in trie:
            break
        transitions = trie[word].items()
        transitions_randomized = {w : random.random() * p for w, p in transitions }
        next_state = max(transitions_randomized.items(), key=lambda x : x[1])
        if verbose:
            print("Current word :: ", word)
            print("Transitions :: ", transitions)
            print("Next state :: ", next_state)
        res.append(word)
        word = next_state[0]
    return res

In [36]:
generated_words = generate2(trie, initial_word='we', max_len=20, verbose=False)

In [37]:
' '.join(generated_words)

'we are the best thing that is a long time for me <SENTENCE> i have not know what you have'